In [5]:
from tensorflow.keras.datasets import mnist
def getMnistData(reshaped=True):
    (train_image,train_label),(test_image,test_labels)=mnist.load_data()
    return train_image,train_label,test_image,test_labels

In [7]:
import numpy as np
class MaxPool2(object):
    def iterate_regions(self, image):
        h, w, _ = image.shape

        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i*2):(i*2 + 2), (j*2):(j * 2 + 2)]
                yield im_region, i, j

    def forward(self, input):
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros(shape=(h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, dL_dout):
        dL_dinput = np.zeros(shape=self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):

            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if im_region[i2, j2, f2] == amax[f2]:
                            dL_dinput[i*2 + i2, j*2 + j2,
                                      f2] = dL_dout[i, j, f2]
        return dL_dinput

In [ ]:
import numpy as np
class Conv3x3(object):
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3, ) / 9

    def iterate_regions(self, image):
        h, w = image.shape
        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:i+3, j:j+3]
                yield im_region, i, j

    def forward(self, input):
        self.last_input = input

        h, w = input.shape

        output = np.zeros(shape=(h-2, w-2, self.num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        return output

    def backprop(self, dL_dout, learning_rate):
        dL_dfilters = np.zeros(self.filters.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                dL_dfilters[f] += dL_dout[i, j, f] * im_region
        self.filters -= learning_rate * dL_dfilters
        return None

In [ ]:
import numpy as np
class Softmax:
    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(shape=nodes)

    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals
        exp = np.exp(totals)

        return exp / np.sum(exp, axis=0)

    def backprop(self, dL_dout, learning_rate):
        for i, gradient in enumerate(dL_dout):
            if gradient == 0:
                continue
            t_exp = np.exp(self.last_totals)
            S = np.sum(t_exp)
            dout_dt = -t_exp[i] * t_exp / S**2
            dout_dt[i] = t_exp[i] * (S - t_exp[i]) / S**2

            dt_dw = self.last_input
            dt_db = 1
            dt_dinputs = self.weights

            dL_dt = gradient * dout_dt

            dL_dw = dt_dw[np.newaxis].T @ dL_dt[np.newaxis]
            dL_db = dL_dt * dt_db
            dL_dinputs = dt_dinputs @ dL_dt

            self.weights -= learning_rate * dL_dw
            self.biases -= learning_rate * dL_db

            return dL_dinputs.reshape(self.last_input_shape)

In [11]:
import numpy as np
train_images, train_labels, test_images, test_labels = getMnistData(
    reshaped=False)
train_images = train_images[:3000]
train_labels = train_labels[:3000]
test_images = test_images[:1000]
test_labels = test_labels[:1000]

conv_layer = Conv3x3(num_filters=8)       
pooling_layer = MaxPool2()                
softmax_layer = Softmax(13*13*8, 10)      


def forward(image, label):
    out = conv_layer.forward((image / 255) - 0.5)
    out = pooling_layer.forward(out)
    out = softmax_layer.forward(out)

    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0
    return out, loss, acc


def train(im, label, lr=.005):
    out, loss, acc = forward(im, label)
    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]
    gradient = softmax_layer.backprop(gradient, lr)
    gradient = pooling_layer.backprop(gradient)
    conv_layer.backprop(gradient, lr)

    return loss, acc


print('MNIST CNN initialized!')
epochs = 10
for epoch in range(epochs):
    print(f"-------Epoch {epoch+1}-------")
    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]

    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        if i > 0 and i % 100 == 99:
            print(
                f'[Step {i}] Past 100 steps: Average Loss {loss/100:.3f} | Accuracy: {num_correct}')

            loss = 0
            num_correct = 0

        l, acc = train(im, label)
        loss += l
        num_correct += acc
print('\n--- Testing the CNN ---')

loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

num_test = len(test_labels)
print(f'Test Loss: {loss / num_test}')
print(f'Test Accuracy: {num_correct / num_test}')

MNIST CNN initialized!
-------Epoch 1-------
[Step 99] Past 100 steps: Average Loss 2.218 | Accuracy: 20
[Step 199] Past 100 steps: Average Loss 1.980 | Accuracy: 37
[Step 299] Past 100 steps: Average Loss 1.653 | Accuracy: 42
[Step 399] Past 100 steps: Average Loss 1.158 | Accuracy: 66
[Step 499] Past 100 steps: Average Loss 0.912 | Accuracy: 69
[Step 599] Past 100 steps: Average Loss 0.965 | Accuracy: 73
[Step 699] Past 100 steps: Average Loss 0.803 | Accuracy: 70
[Step 799] Past 100 steps: Average Loss 0.813 | Accuracy: 77
[Step 899] Past 100 steps: Average Loss 0.474 | Accuracy: 88
[Step 999] Past 100 steps: Average Loss 0.761 | Accuracy: 76
[Step 1099] Past 100 steps: Average Loss 0.580 | Accuracy: 81
[Step 1199] Past 100 steps: Average Loss 0.612 | Accuracy: 83
[Step 1299] Past 100 steps: Average Loss 0.347 | Accuracy: 88
[Step 1399] Past 100 steps: Average Loss 0.565 | Accuracy: 84
[Step 1499] Past 100 steps: Average Loss 0.479 | Accuracy: 85
[Step 1599] Past 100 steps: Average 

KeyboardInterrupt: 